<a href="https://colab.research.google.com/github/kuakeke/113-2pgfinal/blob/main/%E7%A8%8B%E5%BC%8F%E8%AA%9E%E8%A8%80_%E6%9C%9F%E6%9C%AB%E5%B0%88%E9%A1%8C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 113-2程式語言 期末專題

柯昱葶 國文115級 41120122L

程式名稱：試題分析計算機

概要：學習評量的試題分析計算機。供老師計算一份試題的難度、鑑別度、信度。我在網絡上找不到公開的計算機，就想製作一個免費計算機供老師使用。老師將一份批改中的n題ABCD選擇題的試題和正確答案，透過這個程式可以計算出這份試卷的難度、鑑別度、信度，以及可看見k-mean聚類圖表，和AI分析建議。

操作方式：
1. 點擊以下“輸入成績”鏈接。
2. 在第一頁輸入學生們的ABCD作答答案，可自行增加學生（S）和題目（Q）的欄位。
3. 在第二頁輸入正確答案，可自行增加題目（Q）的欄位。
4. 回到程式頁面點擊執行。
5. 此程式會自動下載分析結果，excel檔案和k-mean聚類圖表。

**輸入成績**

https://docs.google.com/spreadsheets/d/1NOiihN2KOGudwPLvpgVpArZwz8vJ4oEUtY1KjfEBiTg/edit?usp=sharing

In [ ]:
import numpy as np
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

In [ ]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

In [ ]:
import pandas as pd
# read data and put it in a dataframe
# 在 google 工作表載入 gsheets
gsheets = gc.open_by_url('https://docs.google.com/spreadsheets/d/1NOiihN2KOGudwPLvpgVpArZwz8vJ4oEUtY1KjfEBiTg/edit?usp=sharing')

In [ ]:
from gspread_dataframe import get_as_dataframe
ws_students = gsheets.get_worksheet(0)
df_students = get_as_dataframe(ws_students, dtype=str).dropna(how='all')
first_col = df_students.columns[0]
df_students = df_students.set_index(first_col)

In [ ]:
ws_answer = gsheets.get_worksheet(1)
df_answer = get_as_dataframe(ws_answer, header=None).dropna(how='all')

In [ ]:
# 取出第2列作為正確答案（跳過題號欄）
answer_key = df_answer.iloc[1, 0:].tolist()  # 第2列（index=1），跳過第1欄

In [ ]:
answer_key

['A',
 'B',
 'C',
 'D',
 'A',
 'B',
 'C',
 'D',
 'A',
 'B',
 'A',
 'B',
 'C',
 'D',
 'A',
 'B',
 'C',
 'D',
 'A',
 'B']

In [ ]:
# 檢查題目數量是否一致
assert df_students.shape[1] == len(answer_key), "題目數與正確答案數不符！"

# 比對學生答案與正確答案
df_result = df_students.eq(answer_key).astype(int)

# 加上總分欄
df_result['Total'] = df_result.sum(axis=1)

# 顯示前幾筆結果
df_result_reset = df_result.reset_index()
df_result_reset

,question,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9,...,Q12,Q13,Q14,Q15,Q16,Q17,Q18,Q19,Q20,Total
0,S1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,0,18
1,S2,1,0,0,0,0,1,0,0,0,...,0,0,0,0,1,0,0,0,0,4
2,S3,1,1,1,1,1,0,0,0,0,...,1,1,1,1,0,0,0,0,0,10
3,S4,1,0,1,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,4
4,S5,0,0,1,1,1,1,1,1,1,...,0,1,1,1,1,1,1,1,0,14
5,S6,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,0,18
6,S7,1,1,1,1,1,1,1,0,0,...,1,1,1,1,1,1,0,0,0,14
7,S8,1,1,1,1,1,0,0,0,0,...,1,1,1,1,0,0,0,0,0,10
8,S9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,2
9,S10,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,2


In [ ]:
from gspread_dataframe import set_with_dataframe

# Open the spreadsheet first
spreadsheet = gc.open_by_url('https://docs.google.com/spreadsheets/d/1NOiihN2KOGudwPLvpgVpArZwz8vJ4oEUtY1KjfEBiTg/edit?usp=sharing')

# 選擇要更新的工作表（選擇第三個工作表，索引為 2）
worksheet = spreadsheet.get_worksheet(2)

# 將 DataFrame 寫入 Google Sheets
set_with_dataframe(worksheet, df_result_reset)

In [ ]:
df = df_result_reset.drop(columns=['Total'])

In [ ]:
df = df.set_index("question")

高分組和低分組

難度計算

In [ ]:
# 計算每位學生總分（或平均）
df["score"] = df.sum(axis=1)

# 依總分排序
sorted_students = df.sort_values("score", ascending=False)
n = len(sorted_students)
group_size = round(n * 0.27)
# 分組
high_group = sorted_students.head(group_size).drop(columns="score")
low_group = sorted_students.tail(group_size).drop(columns="score")

# 計算每題高分組與低分組的正答率
high_correct_rate = high_group.T.mean(axis=1)
low_correct_rate = low_group.T.mean(axis=1)

# 計算每題的難度（高+低平均）
difficulty = (high_correct_rate + low_correct_rate) / 2

# 四捨五入到小數點後兩位
difficulty = difficulty.round(2)

# 評鑑描述函式
def get_evaluation_label(p):
    if p >= 0.80:
        return "極容易"
    elif 0.60 <= p < 0.80:
        return "容易"
    elif 0.40 <= p < 0.60:
        return "難易適中"
    elif 0.20 <= p < 0.40:
        return "困難"
    else:
        return "極困難"

# 套用評鑑描述
P = difficulty.apply(get_evaluation_label)

鑑別度計算

In [ ]:
# 計算每題高分組與低分組的正答率
high_correct_counts = high_group.T.sum(axis=1)
low_correct_counts = low_group.T.sum(axis=1)

# 計算鑑別度
discrimination = (high_correct_counts - low_correct_counts) / (n / 2)

# 四捨五入到小數點後兩位
discrimination = discrimination.round(2)

# 評鑑描述函式
def get_evaluation_label(d):
    if d >= 0.40:
        return "極佳的試題"
    elif 0.30 <= d < 0.40:
        return "尚可的試題，可能需要稍加改進"
    elif 0.20 <= d < 0.30:
        return "不佳的試題，必須加以改進，或棄卻"
    else:
        return "極差的試題，應棄卻"

# 套用評鑑描述
evaluation = discrimination.apply(get_evaluation_label)


In [ ]:
# 合併成 DataFrame
d_df = pd.DataFrame({
    "題號": high_correct_rate.index,
    "高分組答對率": high_correct_rate,
    "高分組答對數": high_correct_counts,
    "低分組答對率": low_correct_rate,
    "低分組答對數": low_correct_counts,
    "難度": difficulty,
    "鑑別度": discrimination,
    "鑑別評鑑": evaluation,
    "難易評鑑": P
})

print(d_df)

      題號    高分組答對率  高分組答對數    低分組答對率  低分組答對數    難度  鑑別度              鑑別評鑑  \
Q1    Q1  1.000000       3  0.333333       1  0.67  0.4             極佳的試題   
Q2    Q2  1.000000       3  0.000000       0  0.50  0.6             極佳的試題   
Q3    Q3  1.000000       3  0.333333       1  0.67  0.4             極佳的試題   
Q4    Q4  1.000000       3  0.000000       0  0.50  0.6             極佳的試題   
Q5    Q5  1.000000       3  0.000000       0  0.50  0.6             極佳的試題   
Q6    Q6  1.000000       3  0.000000       0  0.50  0.6             極佳的試題   
Q7    Q7  1.000000       3  0.000000       0  0.50  0.6             極佳的試題   
Q8    Q8  0.666667       2  0.000000       0  0.33  0.4             極佳的試題   
Q9    Q9  0.666667       2  0.333333       1  0.50  0.2  不佳的試題，必須加以改進，或棄卻   
Q10  Q10  0.000000       0  0.333333       1  0.17 -0.2         極差的試題，應棄卻   
Q11  Q11  1.000000       3  0.333333       1  0.67  0.4             極佳的試題   
Q12  Q12  1.000000       3  0.000000       0  0.50  0.6             極佳的試題   

In [ ]:
from gspread_dataframe import set_with_dataframe

# Open the spreadsheet first
spreadsheet = gc.open_by_url('https://docs.google.com/spreadsheets/d/1NOiihN2KOGudwPLvpgVpArZwz8vJ4oEUtY1KjfEBiTg/edit?usp=sharing')

# 選擇要更新的工作表（選擇第四個工作表，索引為 3）
worksheet = spreadsheet.get_worksheet(3)

# 將 DataFrame 寫入 Google Sheets
set_with_dataframe(worksheet, d_df)



---

用K-mean聚類：難度和鑑別度的關係。

In [ ]:
!pip install -U kaleido

In [ ]:
import plotly.io as pio
from google.colab import files
import pandas as pd
import plotly.express as px
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

# 建立難度與鑑別度平均資料
# Use the index of d_df (which contains question names) for grouping
question_summary = d_df.reset_index().groupby('index')[['難度', '鑑別度']].mean().reset_index()

# Rename the index column to 'question' for clarity
question_summary = question_summary.rename(columns={'index': 'question'})

# 根據難度與鑑別度建立中位數分類
diff_median = question_summary['鑑別度'].median()
difficulty_median = question_summary['難度'].median()

def classify_question(row):
    if row['難度'] >= difficulty_median and row['鑑別度'] >= diff_median:
        return '難度高鑑別高'
    elif row['難度'] >= difficulty_median and row['鑑別度'] < diff_median:
        return '難度高鑑別低'
    elif row['難度'] < difficulty_median and row['鑑別度'] >= diff_median:
        return '難度低鑑別高'
    else:
        return '難度低鑑別低'

# 加入分類欄位
question_summary['分類群組'] = question_summary.apply(classify_question, axis=1)

# 加入 KMeans 聚類
kmeans = KMeans(n_clusters=4, random_state=42)
question_summary['分類群組'] = kmeans.fit_predict(question_summary[['難度', '鑑別度']])

# PCA 降維（可視化用，可不顯示）
pca = PCA(n_components=2)
components = pca.fit_transform(question_summary[['難度', '鑑別度']])
question_summary['主成分1'] = components[:, 0]
question_summary['主成分2'] = components[:, 1]

# 合併相同難度＆鑑別度的題目名稱
grouped = question_summary.groupby(['難度', '鑑別度', '分類群組']).agg({
    'question': lambda x: ', '.join(x.astype(str))
}).reset_index()

# 將合併後資料與原本的分類群組對齊
fig = px.scatter(
    grouped,
    x='難度',
    y='鑑別度',
    color='分類群組',
    text='question',
    title='Question Classification Analysis Chart (Difficulty and Discrimination)',
    labels={'Discrimination': 'Discrimination', 'Difficulty': 'Difficulty'},
    hover_data=['question']
)

texts = [
    "Blue dots: High difficulty + High discrimination (very suitable for student differentiation)",
    "Red dots: High difficulty but low discrimination (may be too hard to differentiate student ability)",
    "Green dots: Low difficulty but high discrimination (easy but effective good questions)",
    "Purple dots: Low difficulty and low discrimination (mostly ineffective questions, should consider revision)"
]

for i, text in enumerate(texts):
    fig.add_annotation(
        text=text,
        xref="paper", yref="paper",
        x=0.5, y=-0.25 - i*0.05,  # 每行往下移 0.05 單位
        showarrow=False,
        align="center",
        font=dict(size=12, family="Microsoft JhengHei"),
        bgcolor="white",
        opacity=0.9
    )

# 美化圖表
fig.update_layout(
    font_family="Microsoft JhengHei",
    xaxis_title="Difficulty",
    yaxis_title="Discrimination",
    legend_title="Cluster Group",
    title_x=0.5,
    height=800,
    width=1000,
    margin=dict(b=200)
)

fig.show()


用k-mean聚類和pca降維，將各題的難度和鑑別度的數據結合並且進行聚類。將每個群組以顏色分類得出的分析結果如下：

藍色點：難度高 + 鑑別高（很適合篩選學生）

紅色點：難度高但鑑別低（可能太難無法分出學生程度）

綠色點：難度低但鑑別高（簡單但有效的好題）

紫色點：難度低且鑑別低（多為無效題目，應考慮修改）

這樣身為老師在完成批改一份試卷後，將學生答案寫入這裡，就會被分析出題目的難度和鑑別度，以快速知道哪些題目是好題或是能淘汰的題目。




---
下載成圖片


In [ ]:
output_file = "聚類分析圖.png"
pio.write_image(fig, output_file, format='png', width=1000, height=800, scale=2)
files.download(output_file)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

信度計算

In [ ]:
# Cronbach's α 計算函數
def cronbach_alpha(df):
    itemscores = df.T
    itemvars = itemscores.var(axis=1, ddof=1)
    totalscore = df.sum(axis=1)
    n_items = df.shape[1]
    return (n_items / (n_items - 1)) * (1 - itemvars.sum() / totalscore.var(ddof=1))

alpha = cronbach_alpha(df)
print(f"信度 = {alpha:.3f}")

信度 = 0.756


α 值區間	解釋

≥ 0.90	非常好（Excellent）

0.80–0.89	良好（Good）

0.70–0.79	可接受（Acceptable）

0.60–0.69	邊緣（Questionable）

< 0.60	不理想（Poor）



---

AI建議

In [ ]:
from google.colab import userdata
import random
import google.generativeai as genai
apikey = userdata.get('GOOGLE_API_KEY')
genai.configure(api_key = apikey)
model = genai.GenerativeModel("gemini-1.5-flash")

In [ ]:
prompt = f"""
這是一份試卷的學習評量，以下數據分析了高分組學生、低分組學生、難度、鑑別度、信度。

{d_df[['題號', '高分組答對率', '高分組答對數', '低分組答對率', '低分組答對數','難度', '鑑別度', '鑑別評鑑', '難易評鑑']].to_string(index=False)}
(f"信度 = {alpha:.3f}")

請你根據這份資料進行以下分析：
1.敘述此班級學生的成績狀況，低分組和高分組學生的數量。
2.找出最難和最簡單的題目，並給出改良建議。
3.根據鑑別度結果，為試卷題目給出建議。
4.這份試卷的信度是多少？請給出分析和建議。

"""

In [ ]:
response = model.generate_content(prompt)
print(response.text)

根據提供的數據，我們可以進行以下分析：

**1. 班級學生成績狀況及組別數量：**

數據顯示，高分組學生在每題都答對了3題(高分組人數為3人)。低分組學生在部分題目中答對了1題，也有一些題目沒有答對(低分組人數為3人)。  因此，總共有6位學生參與此次測驗。  無法得知高低分組的劃分標準，但可以推斷，這是個極端分佈的結果，需要進一步調查了解原因，例如：題目過於簡單或過於困難，導致分數差異過大，也可能與學生學習狀況的差異有關。


**2. 最難和最簡單的題目及改良建議：**

* **最簡單的題目：** Q1, Q3, Q11, Q13。這些題目高分組答對率為100%，低分組答對率也相對較高(33.33%)。  這些題目鑑別度雖然合格(0.4)，但鑑別能力不足，未能有效區分高低分組學生的能力。

    * **改良建議：**  應增加題目的難度和複雜度，例如，可以增加步驟、改變題型(例如從選擇題改為簡答題或計算題)，或增加需要更高階思考能力的元素。 避免只考查基礎知識，應加入應用或分析層級的題目。


* **最難的題目：** Q10 和 Q20。這些題目高分組答對率為0%，低分組答對率也低(33.33%)，鑑別度為負值(-0.2)，表示這些題目無法區分高低分組學生，甚至低分組學生答對率還比高分組高。

    * **改良建議：** 這些題目應該直接刪除。  需要檢視題目本身是否存在錯誤、表達不清或超出教學範圍等問題。  如果題目本身沒有問題，則需要重新設計，降低題目難度，讓它更符合學生的學習程度，並使其具有鑑別度。


**3. 鑑別度結果建議：**

* **鑑別度極佳 (0.6)：** Q2, Q4, Q5, Q6, Q7, Q12, Q14, Q15, Q16, Q17。這些題目高分組答對率高，低分組答對率低，有效區分了高低分組學生的能力。  但需要注意的是，這些題目的高分組答對率均為100%，暗示可能過於簡單，無法有效測量高分組學生的實際能力。

* **鑑別度中等 (0.4)：** Q1, Q3, Q8, Q11, Q13, Q18。這些題目也具備一定的鑑別能力，但鑑別效果不夠理想，建議參考第一點的改良建議進行調整。

* **鑑別度不佳 (0.2)：** Q9, Q19。這些題目鑑別度過低，需要大幅度修改或刪除。  需要分析原因，例如題目設計、

In [ ]:
from gspread_dataframe import set_with_dataframe

spreadsheet = gc.open_by_url('https://docs.google.com/spreadsheets/d/1NOiihN2KOGudwPLvpgVpArZwz8vJ4oEUtY1KjfEBiTg/edit?usp=sharing')

# 選擇要更新的工作表（選擇第五個工作表，索引為 4）
worksheet = spreadsheet.get_worksheet(4)

# 將生成的文字放入一個 DataFrame
response_df = pd.DataFrame([[response.text]])

# 將 DataFrame 寫入 Google Sheets
set_with_dataframe(worksheet, response_df)

In [ ]:
!pip install --upgrade gspread gspread_dataframe openpyxl

ERROR: Operation cancelled by user


In [ ]:
import pandas as pd
from gspread_dataframe import get_as_dataframe
from google.colab import auth
import gspread
from google.auth import default

try:
    auth.authenticate_user()
    creds, _ = default()
    gc = gspread.authorize(creds)
except:
    print("Authentication failed. Please run the authentication cell.")

spreadsheet = gc.open_by_url('https://docs.google.com/spreadsheets/d/1NOiihN2KOGudwPLvpgVpArZwz8vJ4oEUtY1KjfEBiTg/edit?usp=sharing')

all_worksheets = spreadsheet.worksheets()

with pd.ExcelWriter('試卷分析.xlsx', engine='openpyxl') as writer:
    for ws in all_worksheets:
        try:
            df = get_as_dataframe(ws, evaluate_formulas=True)
            df.to_excel(writer, sheet_name=ws.title, index=False)
            print(f"Successfully wrote sheet '{ws.title}' to Excel.")
        except Exception as e:
            print(f"Could not read or write sheet '{ws.title}': {e}")

from google.colab import files
files.download('試卷分析.xlsx')

Successfully wrote sheet '學生' to Excel.
Successfully wrote sheet '正確答案' to Excel.
Successfully wrote sheet '學生對錯' to Excel.
Successfully wrote sheet '鑑別結果' to Excel.
Successfully wrote sheet 'AI建議' to Excel.
Successfully wrote sheet '使用說明' to Excel.
Successfully wrote sheet '範本' to Excel.
Successfully wrote sheet '範本2' to Excel.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
worksheets = spreadsheet.worksheets()

# 清除第3到第5個工作表（index從0開始）
for i in [2, 3, 4]:
    ws = worksheets[i]
    ws.clear()  # 清除該工作表所有儲存格內容

worksheet = spreadsheet.get_worksheet(0)

# 取得目前工作表的最大行列數
num_rows = worksheet.row_count
num_cols = worksheet.col_count

# 產生 B2 到最後一格的範圍（如 B2:Z1000）
from gspread.utils import rowcol_to_a1

# ---------- 第一頁 ----------
ws1 = spreadsheet.get_worksheet(0)  # 第一頁（index 從 0 開始）
num_rows_1 = ws1.row_count
num_cols_1 = ws1.col_count

# 從 B2 到最後的範圍
end_cell_1 = rowcol_to_a1(num_rows_1, num_cols_1)
range1 = f"B2:{end_cell_1}"

# ---------- 第二頁 ----------
ws2 = spreadsheet.get_worksheet(1)  # 第二頁
num_rows_2 = ws2.row_count
num_cols_2 = ws2.col_count

# 從 B1 到最後（整欄清除）
end_cell_2 = rowcol_to_a1(num_rows_2, num_cols_2)
range2 = f"A2:{end_cell_2}"

# ---------- 執行清除 ----------
ws1.batch_clear([range1])
ws2.batch_clear([range2])

print(f"已清除第一頁範圍：{range1}")
print(f"已清除第二頁範圍：{range2}")

已清除第一頁範圍：B2:Z1000
已清除第二頁範圍：A2:Z1000
